## Imports

You will mainly be using [Numpy](https://numpy.org) and [Matplotlib's Pyplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html) library to generate the data and plot the graphs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Plot Utilities

You will be plotting several graphs in this notebook so it's good to have a utility function for that. The following code will visualize numpy arrays into a graph using Pyplot's [plot()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) method. The x-axis will contain the time steps. The exact unit is not critical for this exercise so you can pretend it is either seconds, hours, year, etc. The y-axis will contain the measured values at each time step.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format (string) - line style when plotting the graph
      start (int) - first time step to plot
      end (int) - last time step to plot
      label (list of strings)- tag for the line
    """
    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    # Plot the time series data
    plt.plot(time[start:end], series[start:end], format)
    # Label the x-axis
    plt.xlabel("Time")
    # Label the y-axis
    plt.ylabel("Value")
    if label:
      plt.legend(fontsize=14, labels=label)
    # Overlay a grid on the graph
    plt.grid(True)
    # Draw the graph on screen
    plt.show()

## Trend

The *trend* describes the general tendency of the values to go up or down as time progresses. Given a certain time period, you can see if the graph is following an upward/positive trend, downward/negative trend, or just flat. For instance, the housing prices in a good location can see a general increase in valuation as time passes.

The simplest example to visualize is data that follows a straight line. You will use the function below to generate that. The `slope` argument will determine what the trend is. If you're coming from a mathematics background, you might recognize this as the [slope-intercept form](https://en.wikipedia.org/wiki/Linear_equation#Slope%E2%80%93intercept_form_or_Gradient-intercept_form) with the y-intercept being `0`.

In [ ]:
def trend(time, slope=0):
    """
    Generates synthetic data that follows a straight line given a slope value.

    Args:
      time (array of int) - contains the time steps
      slope (float) - determines the direction and steepness of the line

    Returns:
      series (array of float) - measurements that follow a straight line
    """

    # Compute the linear series given the slope
    series = slope * time

    return series

Here is a time series that trends upward. For a downward trend, simply replace the slope value below with a negative value (e.g. `-0.3`).

In [ ]:
# Generate time steps. Assume 1 per day for one year (365 days)
time = np.arange(365)
# Define the slope (You can revise this)
slope = 0.00
# Generate measurements with the defined slope
series = trend(time, slope)
# Plot the results
plot_series(time, series, label=[f'slope={slope}'])

As you can tell, you don't need machine learning to model this behavior. You can simply solve for the equation of the line and you have the perfect prediction model. Data like this is extremely rare in real world applications though and the trend line is simply used as a guide like the one shown in the [Moore's Law](https://en.wikipedia.org/wiki/Moore%27s_law) example in class.

## Seasonality

Another attribute you may want to look for is seasonality. This refers to a recurring pattern at regular time intervals. For instance, the hourly temperature might oscillate similarly for 10 consecutive days and you can use that to predict the behavior on the next day.

You can use the functions below to generate a time series with a seasonal pattern:

In [ ]:
def seasonal_pattern(season_time):
    """
    Just an arbitrary pattern, you can change it if you wish

    Args:
      season_time (array of float) - contains the measurements per time step

    Returns:
      data_pattern (array of float) -  contains revised measurement values according
                                  to the defined pattern
    """
    # Generate the values using an arbitrary pattern
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))
    return data_pattern
def seasonality(time, period, amplitude=1, phase=0):
    """
    Repeats the same pattern at each period

    Args:
      time (array of int) - contains the time steps
      period (int) - number of time steps before the pattern repeats
      amplitude (int) - peak measured value in a period
      phase (int) - number of time steps to shift the measured values

    Returns:
      data_pattern (array of float) - seasonal data scaled by the defined amplitude
    """
    # Define the measured values per period
    season_time = ((time + phase) % period) / period
    # Generates the seasonal data scaled by the defined amplitude
    data_pattern = amplitude * seasonal_pattern(season_time)
    return data_pattern

The cell below shows the seasonality of the data generated because you can see the pattern every 365 time steps.

In [ ]:
# Generate time steps
time = np.arange(4 * 365 + 1)
# Define the parameters of the seasonal data
period = 365
amplitude = 20
# Generate the seasonal data
series = seasonality(time, period=period, amplitude=amplitude)
# Plot the results
plot_series(time, series)

A time series can also contain both trend and seasonality. For example, the hourly temperature might oscillate regularly in short time frames, but it might show an upward trend if you look at multi-year data.

The example below demonstrates a seasonal pattern with an upward trend:

In [ ]:
# Define seasonal parameters
slope = 0.1
period = 365
amplitude = 40
# Generate the data
series = trend(time, slope) + seasonality(time, period=period, amplitude=amplitude)
# Plot the results
plot_series(time, series)

## Noise

In practice, few real-life time series have such a smooth signal. They usually have some noise riding over that signal. The next cells will show what a noisy signal looks like:

In [ ]:
def noise(time, noise_level=1, seed=None):
    """Generates a normally distributed noisy signal
    Args:
      time (array of int) - contains the time steps
      noise_level (float) - scaling factor for the generated signal
      seed (int) - number generator seed for repeatability

    Returns:
      noise (array of float) - the noisy signal
    """
    # Initialize the random number generator
    rnd = np.random.RandomState(seed)

    # Generate a random number for each time step and scale by the noise level
    noise = rnd.randn(len(time)) * noise_level

    return noise


In [ ]:
# Define noise level
noise_level = 5
# Generate noisy signal
noise_signal = noise(time, noise_level=noise_level, seed=42)
# Plot the results
plot_series(time, noise_signal)

Now let's add this to the time series we generated earlier:

In [ ]:
# Add the noise to the time series
series += noise_signal
# Plot the results
plot_series(time, series)